# The problem
    

Using `abc.ABCmeta` does not work when adding another inheritance...
    
Here is an example:

In [2]:
import abc

class ExceptionErrorsParser(BaseException, metaclass=abc.ABCMeta):

    @abc.abstractmethod
    def parseError(self, error):
        """Parse the error to retrieve a human readable messagge"""
        return

class IrodsException(ExceptionErrorsParser):

    pass

IrodsException()

__main__.IrodsException()

# A possible solution
    

Define a class to make sure that some things can be checked when we create the new object.

In [3]:
class AbstractClass(object):

    _required_methods = []

    def __new__(cls, *p, **k):
        required = cls._required_methods[::]
        for attribute in cls.__dict__:
            # print(attribute, required)
            if attribute in required:
                required.pop(required.index(attribute))

        if len(required) > 0:
            raise NotImplementedError(
                "Class '%s' is missing methods required by the parents: '%s'"
                % (cls.__name__, required) )

        instance = super().__new__(cls)
        return instance

Use the class with multiple inheritance, but as the first parent:

In [4]:
class ExceptionErrorsParser(AbstractClass, BaseException):

    _required_methods = ['parseError']

If the method is missing we get the error:

In [6]:
class IrodsException(ExceptionErrorsParser):
    pass

IrodsException()

NotImplementedError: Class 'IrodsException' is missing methods required by the parents: '['parseError']'

If the method is there, everything goes smoothly:

In [7]:
class IrodsException(ExceptionErrorsParser):
    def parseError(self):
        return True

IrodsException()

__main__.IrodsException()